# And how many people live in your country?

- Input (or paste) the names of the countries in the left textbox so that each name is in its own row.
- Clik the button and population for each of the countries will (well, shall:) be displayed in the right textbox.


In [ ]:
import ipywidgets as ipw
import requests

countries = ipw.Textarea(placeholder="Belgium\nSlovenia\nZimbabwe", layout=ipw.Layout(width="300px", height="250px"))
people = ipw.Textarea(placeholder="", layout=ipw.Layout(width="300px", height="250px"))

def on_click(btn): 
    #Get country names
    country_names = get_country_names()
    
    people.value = ''
    countries_split = countries.value.split('\n')
    print('Replaced names:\n')
    for country in countries_split:
        if country:
            try:
                people.value += (get_country_population(country) + '\n')
            except:
                try:
                    country = replace_name_from_list(country_names, country)
                    people.value += (get_country_population(country) + '\n')
                except:
                    people.value += ('No Value' + '\n')
        else:
            people.value += ('' + '\n')
        
def get_country_population(country):
    try:
        r = requests.get('http://api.population.io:80/1.0/population/'+str(country)+'/2018-12-12/') #watchout: hardcoded date.
        population = str(r.json()['total_population']['population'])
        r.raise_for_status()
        if r.status_code != 200:
            print(r.status_code)
        return population
    except:
        return None

def get_country_names():
    try:
        r = requests.get('http://api.population.io:80/1.0/countries')
        country_names = r.json()['countries']
        r.raise_for_status()
        if r.status_code != 200:
            print(r.status_code)
        return country_names
    except:
        return None
    

def replace_name_from_list(service_names, given_name):
    """
    Try to replace the country name with similar one from the list. If no similar name found in hte list, do nothing.
    """
    for name in service_names:
        given_name1 = given_name.replace("&", "and")
        given_name1 = given_name1[0:-2]
        if given_name1 in name:
            print(given_name + " -> " + name)
            return name   

def mk_btn(description):
    btn = ipw.Button(description=description, layout=ipw.Layout(width="605px"))
    btn.on_click(on_click)
    return btn

row0 = ipw.HBox([countries, people])
row1 = ipw.HBox([mk_btn("Get Population")])

ipw.VBox((row0, row1))

<small>Population is retrieved using "Determine total population for a given country on a given date" service found here: http://api.population.io/#!/population/determineTotalPopulationByDate</small>